In [ ]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import functions as F
import pyspark.sql.types as T

In [ ]:
catalog = dbutils.widgets.get("catalog")
schema_landing = dbutils.widgets.get("schema_landing")
schema_silver = dbutils.widgets.get("schema_silver")
schema_gold = dbutils.widgets.get("schema_gold")
volume = dbutils.widgets.get("volume")

**LOAD DATA**

In [15]:
# ---------------------------
# 1. Load silver Delta table 
# ----------------------------

source_table = f"{catalog}.{schema_silver}.weather_clean"
silver_df = spark.readStream.table(source_table)

**DS GOLD TABLE**

In [16]:
target_table_ds = f"{catalog}.{schema_gold}.ds_weather_hourly"
checkpoint_ds = f"/Volumes/{catalog}/{schema_landing}/{volume}/checkpoints/iot_gold/df_ds"

In [17]:
df_ds = (
    silver_df
    .groupBy(
        F.col("city_id"),
        F.col("name").alias("city_name"),
        F.col("country").alias("country_code"),
        F.window(F.col("local_time"), "1 hour").alias("time_window")
    )
    .agg(
        # --- Temperature Stats ---
        F.round(F.avg("temperature"), 2).alias("temp_mean"),
        F.max("temperature").alias("temp_max"),
        F.min("temperature").alias("temp_min"),
        F.round(F.stddev("temperature"), 3).alias("temp_std"),

        # --- Humidity & Pressure ---
        F.round(F.avg("humidity"), 1).alias("humidity_mean"),
        F.round(F.stddev("humidity"), 2).alias("humidity_std"),
        F.round(F.avg("pressure"), 1).alias("pressure_mean"),

        # --- Wind ---
        F.round(F.avg("windspeed"), 2).alias("wind_speed_mean"),

        # --- Totals ---
        F.sum(F.coalesce("rain_1h", F.lit(0))).alias("rain_total_mm"),
        F.round(F.avg("clouds_all"), 1).alias("clouds_avg_pct")
    )
    # Extract window start/end for proper time alignment
    .withColumn("hour_local_start", F.col("time_window.start"))
    .withColumn("hour_local_end", F.col("time_window.end"))
    .drop("time_window")
)


In [18]:
#complete mode is required for aggregations, but we can optimize by only writing the final results to the target table, and using checkpointing to manage state
(df_ds.writeStream
    .format("delta")
    .outputMode("complete")
    .option("checkpointLocation", checkpoint_ds)
    .trigger(availableNow=True)
    .toTable(target_table_ds)
)


**DS GOLD: Optimize**

In [19]:
spark.sql(f"OPTIMIZE {catalog}.{schema_gold}.ds_weather_hourly")

,path,metrics
0,,"{'numFilesAdded': 0, 'numFilesRemoved': 0, 'filesAdded': {'min': None, 'max': None, 'avg': 0.0, 'totalFiles': 0, 'totalSize': 0}, 'filesRemoved': {'min': None, 'max': None, 'avg': 0.0, 'totalFiles': 0, 'totalSize': 0}, 'partitionsOptimized': 0, 'zOrderStats': None, 'clusteringStats': None, 'numBins': 0, 'numBatches': 0, 'totalConsideredFiles': 0, 'totalFilesSkipped': 0, 'preserveInsertionOrder': True, 'numFilesSkippedToReduceWriteAmplification': 0, 'numBytesSkippedToReduceWriteAmplification': 0, 'startTimeMs': 1770579858558, 'endTimeMs': 1770579859064, 'totalClusterParallelism': 8, 'totalScheduledTasks': 0, 'autoCompactParallelismStats': None, 'deletionVectorStats': {'numDeletionVectorsRemoved': 0, 'numDeletionVectorRowsRemoved': 0}, 'recompressionCodec': None, 'numTableColumns': 15, 'numTableColumnsWithStats': 15, 'totalTaskExecutionTimeMs': 0, 'skippedArchivedFiles': 0, 'clusteringMetrics': None}"
